Write python scripts to answer the following questions:
```
Q1 Who did spend most money for renting?
Q2 Which car does make the most amount of income?
Q3 How many time Erick Gilbert (id=6) rent the cars? (need to query from the name)
Q4 What are the cars rented by Ronald Cunningham? (need to query from the name)
Q5 Between 17/03/2014(inclusive) and 29/03/2014(inclusive), 
    what is the total income of ALL cars in the range of time? (use return date)
Q6 Between 17/03/2014(inclusive) and 29/03/2014(inclusive), (use return date)
    what are the total incomes of EACH car in the range of time?
Q7 Find all rental serveices that has duration less than than 3 days?
Q8 Find all rental serveices that has duration greater than or equal than 2 day?
Q9 Find the total rent duration of EACH car?
Q10 Find the ratio [total_income_of_car]/[car_price] of EACH car?
```

```
Q1 ใครจ่ายเงินค่าเช้ามากที่สุด
Q2 รถคันไหนทำรายได้มากที่สุด
Q3 Erick Gilbert (id=6) ยืมรถไปกี่ครั้ง (need to query from the name)
Q4 Ronald Cunningham ยืมรถคันไหนบ้าง (need to query from the name)
Q5 ช่วงวันที่ 17/03/2014(inclusive) ถึง 29/03/2014(inclusive), 
    หารายได้รวมของรถทุกคันในช่วงเวลานี้
Q6 ช่วงวันที่ 17/03/2014(inclusive) ถึง 29/03/2014(inclusive),
    หารายได้รวมของรถแต่ละคันในช่วงเวลานี้
Q7 การเช่ารถครั้งใดบ้างที่มีระยะเวลายืมน้อยกว่า 3 วัน
Q8 การเช่ารถครั้งใดบ้างที่มีระยะเวลายืมมากกว่าหรือเท่ากับ 2 วัน
Q9 หาระยะเวลารวมของการเช่ารถแต่ละคัน
Q10 หาอัตราส่วน [total_income_of_car]/[car_price] ของรถแต่ละคัน
```

# Q1

In [1]:
from myapp.models import *

In [2]:
q1=Customer.objects.annotate(Sum("rent__cost")).order_by("rent__cost__sum").last()
print "%s %s"%(q1.rent__cost__sum, q1)

1799.95 id: 2, Ronald Cunningham


# Q2

In [3]:
q2=Car.objects.annotate(Sum("rent__cost")).order_by("rent__cost__sum").last()
print "%s %s"%(q2.rent__cost__sum, q2)

2240.00 id: 3, TVR Tuscan


# Q3

In [4]:
q3=Customer.objects.filter(id=6).aggregate(Count("rent"))
print q3

{'rent__count': 3}


# Q4

In [5]:
c=Customer.objects.get(first_name="Ronald", last_name="Cunningham")
print "Q4: %s"%len(c.rent_set.all())

Q4: 4


# Q5

In [27]:
from datetime import datetime
import pytz
utc=pytz.timezone('UTC')
start_date = utc.localize( datetime.strptime('2014-03-17','%Y-%m-%d') )
stop_date = utc.localize( datetime.strptime('2014-03-29','%Y-%m-%d') )
#for i in Rent.objects.filter(return_date__range=[start_date, stop_date]):
#    print i.return_date
Rent.objects.filter(return_date__range=[start_date, stop_date]).aggregate(Sum("cost"))

{'cost__sum': Decimal('8109.25')}

# Q6

In [23]:
car_total_rent={}
for i in Rent.objects.filter(return_date__range=[start_date, stop_date]):
    key=str(i.car.id)+": "+str(i.car.model)
    car_total_rent[key]=car_total_rent.get(key,0)+i.cost

In [24]:
car_total_rent

{'1: L200': Decimal('1529.50'),
 '2: Cooper': Decimal('1525.00'),
 '3: Tuscan': Decimal('2240.00'),
 '4: Z3': Decimal('1119.95'),
 '5: Celica': Decimal('480.00'),
 '6: TT': Decimal('699.95'),
 '7: E320': Decimal('514.85')}

# Q7

In [9]:
count=0
for r in Rent.objects.all():
    dur=(r.return_date-r.rent_date).days
    if 0<=dur and dur<3:
        count+=1
        print "rent:%s, return:%s"%(r.rent_date, r.return_date)
        print "%s"%r

rent:2014-03-20 00:00:00+00:00, return:2014-03-21 00:00:00+00:00
id: 5
rent:2014-03-20 00:00:00+00:00, return:2014-03-20 00:00:00+00:00
id: 6
rent:2014-03-20 00:00:00+00:00, return:2014-03-21 00:00:00+00:00
id: 7
rent:2014-03-29 00:00:00+00:00, return:2014-03-29 00:00:00+00:00
id: 10
rent:2014-03-29 00:00:00+00:00, return:2014-03-29 00:00:00+00:00
id: 11
rent:2014-03-29 00:00:00+00:00, return:2014-03-29 00:00:00+00:00
id: 12


# Q8

In [10]:
count=0
for r in Rent.objects.all():
    dur=(r.return_date-r.rent_date).days
    if 2<=dur:
        count+=1
        print "rent:%s, return:%s"%(r.rent_date, r.return_date)
        print "%s"%r

rent:2014-03-12 00:00:00+00:00, return:2014-03-17 00:00:00+00:00
id: 1
rent:2014-03-12 00:00:00+00:00, return:2014-03-20 00:00:00+00:00
id: 2
rent:2014-03-13 00:00:00+00:00, return:2014-03-20 00:00:00+00:00
id: 3
rent:2014-03-17 00:00:00+00:00, return:2014-03-20 00:00:00+00:00
id: 4
rent:2014-03-21 00:00:00+00:00, return:2014-03-24 00:00:00+00:00
id: 8
rent:2014-03-24 00:00:00+00:00, return:2014-03-29 00:00:00+00:00
id: 9


# Q9

In [11]:
car_use={}
for r in Rent.objects.all():
    dur=(r.return_date-r.rent_date).days
    car_use[r.car.model]=car_use.get(r.car.model,0)+dur
print car_use

{u'L200': 9, u'Cooper': 11, u'E320': 3, u'TT': 0, u'Celica': -1, u'Tuscan': 6, u'Z3': 0}


In [12]:
car_use={}
for r in Rent.objects.all():
    dur=(r.return_date-r.rent_date).days
    if dur<0:
        dur=0
    car_use[r.car.model]=car_use.get(r.car.model,0)+dur
print car_use

{u'L200': 10, u'Cooper': 11, u'E320': 3, u'TT': 1, u'Celica': 0, u'Tuscan': 7, u'Z3': 1}


# Q10

In [13]:
q=Car.objects.annotate(Sum("rent__cost"))

In [14]:
for i in q:
    print "%.3f %s"%(i.rent__cost__sum/i.price, i)

0.153 id: 1, Mitsubishi L200
0.122 id: 2, Mini Cooper
0.124 id: 3, TVR Tuscan
0.080 id: 4, BMW Z3
0.103 id: 5, Toyota Celica
0.032 id: 6, Audi TT
0.033 id: 7, Mercedes E320


In [25]:
Rent.objects.all()

[<Rent: id: 1>, <Rent: id: 2>, <Rent: id: 3>, <Rent: id: 4>, <Rent: id: 5>, <Rent: id: 6>, <Rent: id: 7>, <Rent: id: 8>, <Rent: id: 9>, <Rent: id: 10>, <Rent: id: 11>, <Rent: id: 12>, <Rent: id: 13>, <Rent: id: 14>, <Rent: id: 15>, <Rent: id: 16>, <Rent: id: 17>]